# <div style="text-align:center"> DS7333 - Case Study 6 | SPAM</div>
### <div style="text-align:center">Andy Ho, An Nguyen, Jodi Pafford</div>
<div style="text-align:center">June 17, 2019</div>

# 1 Introduction

This case study will answer Question 20 in Chapter 3 of "Data Science in R: A Case Studies Approach to Computational Reasoning and Problem Solving" (Nolan and Lang). 

>Question 20 is: "In the section called “Classifying New Messages” we used the test set that we had put aside to both select , the threshold for the log odds, and to evaluate the Type I and II errors incurred when we use this threshold. Ideally, we choose from another set of messages that is both independent of our training data and our test data. The method of cross-validation is designed to use the training set for training and validating the model. Implement 5-fold cross-validation to choose and assess the error rate with our training data. To do this, follow the steps: 
    <br> 
    <br> A.) Use the sample() function to permute the indices of the training set, and organize these permuted indices into 5 equal-size sets, called folds. 
    <br> 
    <br> B.) For each fold, take the corresponding subset from the training data to use as a 'test' set. Use the remaining messages in the training data as the training set. Apply the functions developed in the section called “Implementing the Naïve Bayes Classifier” to estimate the probabilities that a word occurs in a message given it is spam or ham, and use these probabilities to compute the log likelihood ratio for the messages in the training set.
    <br> 
    <br> C.) Pool all of the LLR values from the messages in all of the folds, i.e., from all of the training data, and use these values and the typeIErrorRate() function to select a threshold that achieves a 1% Type I error.
    <br> 
    <br> D.) Apply this threshold to our original/real test set and find its Type I and Type II errors."

The data was extracted, cleaned, reformatted and prepared for analysis.  The preparation includes parsing out words from the body of emails and removing stop words. Once the data was prepared, we set aside one third of the data to be used as a validation set.  The remaining two third of the data used as the training set.  The training set was further divided into 5 folds for cross-validation of the Naive-Bayes classification method in determining spam and non-spam emails.

# 2 Background

Spam messages have been sent to email accounts almost since the inception of emails. People have gotten good at recognizing Spam messages, however AI should be able to create filters to also recognize Spam. Doing so would save companies from the risk of employees who aren't as 'savy' clicking links inside Spam emails, potentially saving the company time and money. 

Before an AI can learn to classify messages, programmers (people) must do some basic analysis to find the right parameters to train the AI. Does the computer simply need to look at the sending email address, or does it also need to include the entire message? How many Type I and Type II errors are found in doing any kind of filtering?

# 3 Methods

Following the code from Chapter 3 Sections 3.1 - 3.5, we cleaned the email files. The first step was removing the stop words and punctuation. Once this was complete, we were able to split the messages into Ham and Spam and combine into one file. The final step in preparation was to create a test and train set in order to complete the steps of the assignment.

# 4 Results

#### A)
To create the 5 folds we first created two 5-element lists, one for spam emails and one for ham (non-spam emails).  We used the sample() function to randomly sample ```r floor(trainNumSpam/fold) ``` [1] for the spam emails and ```r floor(trainNumSpam/fold) ``` [2] in the ham emails training set.  With each new fold the index that have already been allocated is removed from the elements from which sample() is choosing from.

[1]

In [104]:
#run appendix code first
floor(trainNumSpam/fold)

[1] 319

[2]

In [105]:
#run appendix code first
floor(trainNumSpam/fold)

[1] 319

#### B)
Once the indexes have been created. For each fold the indexes are then used to create a list of words from the designated test emails, labeling them as ham or spam. The remaining words are used to create a frequency table from which the likelihood-ratio(LLR) are calculated using the words in test set. This process is repeated over the 5 folds resulting in a list of 5 lists of LLR.

#### C)
We then determine the error rate for each word. For each fold all the LLR with error greater than 1% is deteremined. The tau is the minimum LLR from this set. Once all five taus are found we averaged.

Setting the Type I error rate at 1%, we found a tau of 387.465811932302.

#### D)
The Type I and II error is calculated by applying the validation test set to each of the five training set created above. The error rates are determined and then averaged together.

When applied to the validation data set we see that the Type I error is indeed 1% and the Type II error is 8%.

# 5 Conclusion

### References
+ Nolan, D. and Lang, D. T. “Data Science in R.” CRC Press, 2015 (Chapter 3)
+ http://rdatasciencecases.org/
+ https://rstudio-pubs-static.s3.amazonaws.com/351788_b8d5de284dd645a1b920b7bd77e0967b.html

# 6 Appendix - Code

Ok so you can change the dir structure, but this code does depend on data being in certain directories:


`This Notebook`

`    < spam path folder > `

        messages
    
            easy_ham
        
            spam
        
            etc....
        `
`

In [106]:
#load files
spamPath = "./SpamAssassinMessages/"
#List of messages
dirNames = list.files(path = paste(spamPath, "messages",  sep = .Platform$file.sep))
#Findn file names and place in directories
fullDirNames = paste(spamPath, "messages", dirNames, sep = .Platform$file.sep)

In [107]:
#Function to split message into 2 character vectors (header and body)
splitMessage = function(msg) {
  splitPoint = match("", msg)
  header = msg[1:(splitPoint-1)]
  body = msg[ -(1:splitPoint) ]
  return(list(header = header, body = body))
}

#find the boundary string
getBoundary = function(header) {
  boundaryIdx = grep("boundary=", header)
  boundary = gsub('"', "", header[boundaryIdx])
  gsub(".*boundary= *([^;]*);?.*", "\\1", boundary)
}

#drop attachment using boundary info above
dropAttach = function(body, boundary){
  
  bString = paste("--", boundary, sep = "")
  bStringLocs = which(bString == body)
  
  if (length(bStringLocs) <= 1) return(body)
  
  eString = paste("--", boundary, "--", sep = "")
  eStringLoc = which(eString == body)
  if (length(eStringLoc) == 0) 
    return(body[ (bStringLocs[1] + 1) : (bStringLocs[2] - 1)])
  
  n = length(body)
  if (eStringLoc < n) 
     return( body[ c( (bStringLocs[1] + 1) : (bStringLocs[2] - 1), 
                    ( (eStringLoc + 1) : n )) ] )
  
  return( body[ (bStringLocs[1] + 1) : (bStringLocs[2] - 1) ])
}

In [108]:
library(tm)

stopWords = stopwords()

#clean to stop words to remove case and punctuation
cleanSW = tolower(gsub("[[:punct:]0-9[:blank:]]+", " ", stopWords))

#divide stop words into strings into words by splitting the string on blanks
SWords = unlist(strsplit(cleanSW, "[[:blank:]]+"))

#Drop one-letter stop words
SWords = SWords[ nchar(SWords) > 1 ]

stopWords = unique(SWords)

In [109]:
#final clean text code (like above but in a function for the message words)
cleanText =
function(msg)   {
  tolower(gsub("[[:punct:]0-9[:space:][:blank:]]+", " ", msg))
}

findMsgWords = 
function(msg, stopWords) {
 if(is.null(msg))
  return(character())

 words = unique(unlist(strsplit(cleanText(msg), "[[:blank:]\t]+")))
 
 # drop empty and 1 letter words
 words = words[ nchar(words) > 1]
 words = words[ !( words %in% stopWords) ]
 invisible(words)
}

In [110]:
#Cleaning the entire group of messages in one larger function that contains the functions of the others
processAllWords = function(dirName, stopWords)
{
       # read all files in the directory
  fileNames = list.files(dirName, full.names = TRUE)
       # drop files that are not email, i.e., cmds
  notEmail = grep("cmds$", fileNames)
  if ( length(notEmail) > 0) fileNames = fileNames[ - notEmail ]

  messages = lapply(fileNames, readLines, encoding = "latin1")
  
       # split header and body
  emailSplit = lapply(messages, splitMessage)
       # put body and header in own lists
  bodyList = lapply(emailSplit, function(msg) msg$body)
  headerList = lapply(emailSplit, function(msg) msg$header)
  rm(emailSplit)
  
       # determine which messages have attachments
  hasAttach = sapply(headerList, function(header) {
    CTloc = grep("Content-Type", header)
    if (length(CTloc) == 0) return(0)
    multi = grep("multi", tolower(header[CTloc])) 
    if (length(multi) == 0) return(0)
    multi
  })
  
  hasAttach = which(hasAttach > 0)
  
       # find boundary strings for messages with attachments
  boundaries = sapply(headerList[hasAttach], getBoundary)
  
       # drop attachments from message body
  bodyList[hasAttach] = mapply(dropAttach, bodyList[hasAttach], 
                               boundaries, SIMPLIFY = FALSE)
  
       # extract words from body
  msgWordsList = lapply(bodyList, findMsgWords, stopWords)
  
  invisible(msgWordsList)
}

In [111]:
#apply the 'processAllWords to the entire directory
msgWordsList = lapply(fullDirNames, processAllWords, 
                      stopWords = stopWords) 

Warning message in FUN(X[[i]], ...):
"incomplete final line found on './SpamAssassinMessages//messages/spam/00136.faa39d8e816c70f23b4bb8758d8a74f0'"Warning message in FUN(X[[i]], ...):
"incomplete final line found on './SpamAssassinMessages//messages/spam/0143.260a940290dcb61f9327b224a368d4af'"

In [112]:
#vector of the number of elements in each list
numMsgs = sapply(msgWordsList, length)
numMsgs

[1] 5051 1400  500 1000 1397

In [113]:
#notating which is spam and which is ham
isSpam = rep(c(FALSE, FALSE, FALSE, TRUE, TRUE), numMsgs)

#one list of all words (all 5 files combined)
msgWordsList = unlist(msgWordsList, recursive = FALSE)

#number of emails
numEmail = length(isSpam)
#number of spam emails
numSpam = sum(isSpam)
#number of ham emails
numHam = numEmail - numSpam

set.seed(418910)

#determine indices
validationSpamIdx = sample(numSpam, size = floor(numSpam/3))
validationHamIdx = sample(numHam, size = floor(numHam/3))

#select word vectors
validationMsgWords = c((msgWordsList[isSpam])[validationSpamIdx],
                 (msgWordsList[!isSpam])[validationHamIdx] )
trainingMsgWords = c((msgWordsList[isSpam])[ - validationSpamIdx], 
                  (msgWordsList[!isSpam])[ - validationHamIdx])

#create test (validation) and train
validationIsSpam = rep(c(TRUE, FALSE), 
                 c(length(validationSpamIdx), length(validationHamIdx)))
trainingIsSpam = rep(c(TRUE, FALSE), 
                 c(numSpam - length(validationSpamIdx), 
                   numHam - length(validationHamIdx)))

In [114]:
print(paste("Total emails processed is ", length(msgWordsList), 
           " with ", numSpam, " spam emails and ", numHam, " ham emails.", sep=""))
print(paste(length(validationMsgWords), " emails is set aside to validate the model.", sep=""))
print(paste("The ", length(trainingMsgWords), " emails left is for training the Navie Bayes model.", sep=""))


[1] "Total emails processed is 9348 with 2397 spam emails and 6951 ham emails."
[1] "3116 emails is set aside to validate the model."
[1] "The 6232 emails left is for training the Navie Bayes model."


In [115]:

#number of folds to produce
fold = 5

trainNumEmail = length(trainingIsSpam)
trainNumSpam = sum(trainingIsSpam)
trainNumHam = trainNumEmail - trainNumSpam

#initialize lists of indexes
foldSpamIdx = list()
foldHamIdx = list()

#create a list of x folds holding a list of indexes
#sample() will not sample indexes that have already been taken for previous folds
for (x in 1:fold){
    foldSpamIdx[x] = list(sample((1:trainNumSpam)[!((1:trainNumSpam) %in% unlist(foldSpamIdx))], size = floor(trainNumSpam/fold)))
    foldHamIdx[x] = list(sample((1:trainNumHam)[!((1:trainNumHam) %in% unlist(foldHamIdx))], size = floor(trainNumHam/fold)))
}

In [116]:
#list of indexes after being split
print("The indexes for the spam emails from the training set in each folds are:")
print(foldSpamIdx)

print("The indexes for the ham emails from the training set in each folds are:")
print(foldHamIdx)

[1] "The indexes for the spam emails from the training set in each folds are:"
[[1]]
  [1]  589  602  143  190 1496  149  537  492  351 1299 1025  766  405  814  746
 [16]   98 1383  588   81  181 1297    6  233 1217 1337 1488  757 1387  516  109
 [31] 1412  253  465  481  747 1142 1317 1030  742 1166  565 1476  259  511 1593
 [46] 1266  953 1190 1012 1181  909 1438  969  633  385 1041  786   96 1459 1529
 [61] 1473   92 1262  813  596 1591 1457  186 1479 1322 1516  392 1043 1058 1047
 [76]  232  439 1170  533 1341  231  396 1174  138 1501 1460  257  569 1225  554
 [91]  108 1423  291 1001  472   16  464  169 1069 1561 1382 1008  594 1421 1130
[106] 1087   82  686  220 1543  833 1128  906  837 1120   12 1405  111  176  522
[121]  770 1315  240  716 1454  750  485 1207  871  206  121  418 1248  972  977
[136] 1223  486  788 1197  445  325  184  490  576  851  745  831  713   60  587
[151] 1281 1366   25 1517  921 1062 1136 1245 1052  163  601  879   83  549  452
[166] 1227   93  557 109

[1] "The indexes for the ham emails from the training set in each folds are:"
[[1]]
  [1] 2257  815  233  177  630  931 4526 4611 4083 3058 3573 4605 3050  955 2281
 [16] 4149  485   43 3937 2707 3740 3400 2212 1890  380 1324 2087 3401 1206 3981
 [31] 3566 3749 2632 1413 2476 4119  800 2228 2274 1385 3994 3020 3881 2891 4031
 [46] 2254  550 1003 1729 3233 3214  237 2822 2135 3829 3924 2617 4457  320 1936
 [61] 1947 4109 1987  136 2162 1006 2577 1626  708 3098 1274 2062 1473  109 1444
 [76] 2131 4249 4128 3244  911 1347 4602 3949 3757 2005   86 1195  292 3221 2576
 [91] 2214 4615 2532  741   12 3537 2975 4593  278  648 2835 4556 4006  568 1089
[106] 4497 3078 2976 3892 2984 1088 4164 1820 3102 3806  298 1971 3387  788 1077
[121] 4554 1020 4268 1357 3493 1492 1164  844 1775 2112 4529 2894 2739 2333 2743
[136]  797 2365 3797 1976 2048 2751 1014  451 3746 2204 1498 3070 1952 4177 2519
[151] 2181 2811 2836 2626 3983 1754 2919 1891  749 2899 2731  435  356 1466 1841
[166] 1304  874 2940 4487

In [117]:
#Function to form a matrix of log likelihood rations and porportions
computeFreqs =
function(wordsList, spam, bow = unique(unlist(wordsList)))
{
   # create a matrix for spam, ham, and log odds
  wordTable = matrix(0.5, nrow = 4, ncol = length(bow), 
                     dimnames = list(c("spam", "ham", 
                                        "presentLogOdds", 
                                        "absentLogOdds"),  bow))

   # For each spam message, add 1 to counts for words in message
  counts.spam = table(unlist(lapply(wordsList[spam], unique)))
  wordTable["spam", names(counts.spam)] = counts.spam + .5

   # Similarly for ham messages
  counts.ham = table(unlist(lapply(wordsList[!spam], unique)))  
  wordTable["ham", names(counts.ham)] = counts.ham + .5  


   # Find the total number of spam and ham
  numSpam = sum(spam)
  numHam = length(spam) - numSpam

   # Prob(word|spam) and Prob(word | ham)
  wordTable["spam", ] = wordTable["spam", ]/(numSpam + .5)
  wordTable["ham", ] = wordTable["ham", ]/(numHam + .5)
  
   # log odds
  wordTable["presentLogOdds", ] = 
     log(wordTable["spam",]) - log(wordTable["ham", ])
  wordTable["absentLogOdds", ] = 
     log((1 - wordTable["spam", ])) - log((1 - wordTable["ham", ]))

  invisible(wordTable)
}

In [118]:
#create a list of word table, 1 for each fold as the test set
trainTable = list()
testMsgWords = list()
testIsSpam = list()
trainIsSpam = list()

for (x in 1:fold){
    testSpamIdx = foldSpamIdx[[x]]
    testHamIdx = foldHamIdx[[x]]

    testMsgWords[[x]] = c((trainingMsgWords[trainingIsSpam])[testSpamIdx],
                     (trainingMsgWords[!trainingIsSpam])[testHamIdx] )
    trainMsgWords = c((trainingMsgWords[trainingIsSpam])[ - testSpamIdx], 
                      (trainingMsgWords[!trainingIsSpam])[ - testHamIdx])

    testIsSpam[[x]] = rep(c(TRUE, FALSE), 
                     c(length(testSpamIdx), length(testHamIdx)))
    trainIsSpam[[x]] = rep(c(TRUE, FALSE), 
                     c(numSpam - length(testSpamIdx), 
                       numHam - length(testHamIdx)))

    trainTable[[x]] = list(computeFreqs(trainMsgWords, trainIsSpam[[x]]))
}

In [119]:
#list of words from test and train emails
print("Sample of words from first 5 emails in each fold.")
for (i in 1:fold){
    print(paste("Fold ", i, ":", sep=""))
    for (j in 1:5){
        print(paste("Email ", j, ":", sep=""))
        print(head(testMsgWords[[i]][[j]]))
    }
}

[1] "Sample of words from first 5 emails in each fold."
[1] "Fold 1:"
[1] "Email 1:"
[1] "take"     "control"  "computer" "top"      "line"     "software"
[1] "Email 2:"
[1] "message" "mime"    "format"  "since"   "mail"    "reader" 
[1] "Email 3:"
[1] "help"    "wanted"  "year"    "old"     "fortune" "company"
[1] "Email 4:"
[1] "multi"    "part"     "message"  "mime"     "format"   "nextpart"
[1] "Email 5:"
[1] "content" "type"    "text"    "plain"   "charset" "iso"    
[1] "Fold 2:"
[1] "Email 1:"
[1] "hotels"   "etc"      "giving"   "away"     "travel"   "packages"
[1] "Email 2:"
[1] "html"  "head"  "title" "meta"  "http"  "equiv"
[1] "Email 3:"
[1] "html"  "head"  "body"  "font"  "size"  "color"
[1] "Email 4:"
[1] "dear"       "sir"        "madam"      "well"       "confident" 
[6] "capability"
[1] "Email 5:"
[1] "help"    "wanted"  "year"    "old"     "fortune" "company"
[1] "Fold 3:"
[1] "Email 1:"
[1] "delightful"    "garden"        "ornaments"     "combine"      
[5] "finest" 

In [120]:
#frequency tables from train bag of words
print("First and last 4 elements of the frequency table created from each fold:")
for (i in 1:fold){
    print(paste("Fold ", i, ":", sep=""))
    print(trainTable[[i]][[1]][,1:4])
    columns = ncol(trainTable[[i]][[1]])
    print(trainTable[[i]][[1]][,(columns-3):columns])
    }


[1] "First and last 4 elements of the frequency table created from each fold:"
[1] "Fold 1:"
                    doctype       html      public          dtd
spam            0.037767621  0.3601155  0.07721915  0.037286505
ham             0.003070285  0.1096175  0.02929217  0.003236246
presentLogOdds  2.509681876  1.1894281  0.96932705  2.444217455
absentLogOdds  -0.035424292 -0.3303635 -0.05063375 -0.034757930
                         dak   collectives           tsx           unc
spam            2.405581e-04  2.405581e-04  2.405581e-04  2.405581e-04
ham             2.489420e-04  2.489420e-04  2.489420e-04  2.489420e-04
presentLogOdds -3.425830e-02 -3.425830e-02 -3.425830e-02 -3.425830e-02
absentLogOdds   8.385954e-06  8.385954e-06  8.385954e-06  8.385954e-06
[1] "Fold 2:"
                    doctype       html      public          dtd
spam            0.032475343  0.3634833  0.07240799  0.032956459
ham             0.002240478  0.1117750  0.02597295  0.002572401
presentLogOdds  2.67379189

In [121]:
#Function to calculate the log likelihood ration (LLR)
computeMsgLLR = function(words, freqTable) 
{
       # Discards words not in training data.
  words = words[!is.na(match(words, colnames(freqTable)))]

       # Find which words are present
  present = colnames(freqTable) %in% words

  sum(freqTable["presentLogOdds", present]) +
    sum(freqTable["absentLogOdds", !present])
}

In [122]:
#Calculate Log Likelihood Ratio (LLR) for Test Messages
testLLR = list()
for (x in 1:fold){
    testLLR[[x]] = sapply(testMsgWords[[x]], computeMsgLLR, trainTable[[x]][[1]])
}

In [123]:
#LLR table of training set
print("Example Log Likelihood Ratio calculated for each email:")
for (i in 1:fold){
    print(paste("Fold ", i, ":", sep=""))
    print(head(testLLR[[i]]))
}

[1] "Example Log Likelihood Ratio calculated for each email:"
[1] "Fold 1:"
[1]  129.557422  176.416225   -1.614447 1028.493774  -61.917135  165.878537
[1] "Fold 2:"
[1]   5.255437 536.022763  30.148824 162.363073 -15.600271  -3.357869
[1] "Fold 3:"
[1]  43.77239 309.70896 191.22547  41.40717 115.79050 -25.23422
[1] "Fold 4:"
[1] -47.04663   6.75909  38.80926  70.80801 -58.45018 -62.44841
[1] "Fold 5:"
[1] 310.37104 525.19208 -13.68868 160.70792 973.75839 135.58726


In [124]:
#finding the Type I Error Rate (how many ham are classified as Spam)
typeIErrorRates = 
function(llrVals, isSpam) 
{
  o = order(llrVals)
  llrVals =  llrVals[o]
  isSpam = isSpam[o]

  idx = which(!isSpam)
  N = length(idx)
  list(error = (N:1)/N, values = llrVals[idx])
}

In [125]:
xI = list()
tau01 = list()

for (x in 1:fold){
    xI[[x]] = typeIErrorRates(testLLR[[x]], testIsSpam[[x]])
    tau01[[x]] = min(xI[[x]]$values[xI[[x]]$error <= 0.01])
}

mean_tau01 = mean(unlist(tau01))
mean_tau01


[1] 387.4658

In [126]:
validationLLR = list()
for (x in 1:fold){
    validationLLR[[x]] = sapply(validationMsgWords, computeMsgLLR, trainTable[[x]][[1]])
}

In [127]:
#LLR of validation set
print("Example Log Likelihood Ratio calculated for each email in validation set when applied to training frequency table from each fold:")
for (i in 1:fold){
    print(paste("Fold ", i, ":", sep=""))
    print(head(validationLLR[[i]]))
}

[1] "Example Log Likelihood Ratio calculated for each email in validation set when applied to training frequency table from each fold:"
[1] "Fold 1:"
[1] 483.41879 116.10814 129.08469  33.39559  70.15033 -46.56542
[1] "Fold 2:"
[1] 487.71920 133.71573 119.83349  37.91323  71.12520 -43.07872
[1] "Fold 3:"
[1] 459.68413 130.07999 131.09648  38.50427  74.14865 -38.27852
[1] "Fold 4:"
[1] 434.82839 140.65543 114.68503  30.13731  67.29047 -45.40777
[1] "Fold 5:"
[1] 472.90450 127.80886 128.78708  35.80556  68.89236 -43.10967


In [128]:
# Type I and Type II error rate
typeIErrorRate = 
function(tau, llrVals, spam)
{
  classify = llrVals > tau
  sum(classify & !spam)/sum(!spam)
}

typeIIErrorRate = 
function(tau, llrVals, spam)
{
  classify = llrVals > tau
  sum(classify & spam)/sum(spam)
}

In [129]:
tIerror = list()
tIIerror = list()

for (x in 1:fold){
    tIerror[[x]] = typeIErrorRate(mean_tau01, validationLLR[[x]], validationIsSpam) #1 if all is classify as spam
    tIIerror[[x]] = typeIIErrorRate(mean_tau01, validationLLR[[x]], validationIsSpam) #1 if all is classify as ham
}

print(paste("Type I error rate with tau = ", round(mean_tau01, 2), " is ", round(mean(unlist(tIerror))*100), "%.", sep=""))
print(paste("Type II error rate with tau = ", round(mean_tau01, 2), " is ", round(mean(unlist(tIIerror))*100), "%.", sep=""))


[1] "Type I error rate with tau = 387.47 is 1%."
[1] "Type II error rate with tau = 387.47 is 8%."
